### Домашнее задание 5 - 10 баллов

В этом задании вам предстоит дообучить трансформерную модель для задачи классификации с помощью различных техник и сравнить их между собой.

Датасет: [dair-ai/emotion](https://huggingface.co/datasets/dair-ai/emotion) 

Модель: [google-bert/bert-base-uncased](https://huggingface.co/google-bert/bert-base-uncased) (если хочется, можно заменить на что-то более интересное)

1. Скачайте датасет и модель. Измерьте базовые метрики классификации перед началом экспериментов.

**NB!** Для всех типов дообучения замерьте :
- качество классификации на выходе
- время дообучения
- количество параметров для обучения
- потребление ресурсов (не нужно заморачиваться с профайлингом - можно просто посмотреть в `nvidia-smi` или `torch.cuda.memory_allocated`)

2. Обучите модель в режиме full finetuning - **1 балл**
3. Обучите модель в режиме linear probing - реализуйте кастомную классификационную голову и обучайте только ее. Не забудьте описать, чем обусловлено устройство головы, как вы пришли к такой архитектуре - **2 балла**
4. Обучите модель в режиме PEFT с использованием [prompt tuning или prefix tuning](https://ericwiener.github.io/ai-notes/AI-Notes/Large-Language-Models/Prompt-Tuning-and-Prefix-Tuning). При выборе метода напишите пару слов, почему решили остановиться именно на этом методе - **2 балла**
4. Обучите модель в режиме PEFT с использованием LoRA. Попробуйте подобрать оптимальный ранг - `r`, при желании поэкспериментируйте с остальными гиперпараметрами. Опишите, чем обусловлена ваша финальная конфигурация - **2 балла**

5. Соберите все результаты отдельных замеров в таблицу и сделайте выводы о вычислительной сложности методов, итоговом качестве и прочих наблюдаемых свойствах моделей - **1 балл**

**Общее**

- Принимаемые решения обоснованы (почему выбрана определенная архитектура/гиперпараметр/оптимизатор/преобразование и т.п.) - **1 балл**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**

In [1]:
import time
import random
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
import tabulate
import numpy as np
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings("ignore")

### 1. Загрузим датасет и модель. Измерим базовые метрики классификации перед началом экспериментов.

In [2]:
SEED = 22

def seed_all(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.benchmark = True
        torch.backends.cudnn.deterministic = False


seed_all(SEED)

In [3]:
model_id = "google-bert/bert-base-uncased"
dataset_id = "dair-ai/emotion"

dataset = load_dataset(dataset_id)
labels = dataset["train"].features["label"].names

tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
print(dataset)
print(f"All classes: {labels}")

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})
All classes: ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']


In [8]:
from transformers import TrainerCallback

def tokenize_data(examples):
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128
    )


# Токенизируем данные
tokenized_dataset = dataset.map(tokenize_data, batched=True)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

In [22]:
import psutil

def compute_metrics(eval_pred):
    logits, true_labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    if torch.cuda.is_available():
            gpu_mem_mb = torch.cuda.memory_allocated() / 1e6
    else:
        gpu_mem_mb = 0.0
    
    process = psutil.Process()
    cpu_mem_mb = process.memory_info().rss / 1e6

    return {
        "accuracy": accuracy_score(true_labels, predictions),
        **classification_report(
            true_labels, 
            predictions,
            target_names=labels,
            output_dict=True,
            zero_division=0
        )["macro avg"],
        "gpu_mem_mb": gpu_mem_mb,
        "cpu_mem_mb": cpu_mem_mb
    }

In [23]:
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(labels)
)

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {trainable_params:,}")

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_steps=200,
    weight_decay=0.01,
    metric_for_best_model="f1-score",
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total trainable parameters: 109,486,854


In [24]:
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     1.8489 |
+-------------------------+------------+
| eval_accuracy           |     0.1360 |
+-------------------------+------------+
| eval_precision          |     0.0404 |
+-------------------------+------------+
| eval_recall             |     0.1839 |
+-------------------------+------------+
| eval_f1-score           |     0.0653 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_gpu_mem_mb         |   887.1629 |
+-------------------------+------------+
| eval_cpu_mem_mb         |  2618.8186 |
+-------------------------+------------+
| eval_runtime            |     1.5341 |
+-------------------------+------------+
| eval_samples_per_second |  1303.6770 |
+-------------------------+------------+
| eval_steps_per_second   |    13.6890 |
+---------------

### 2. Обучим модель в режиме full finetuning 

In [25]:
trainer.train()

results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score,Support,Gpu Mem Mb,Cpu Mem Mb
200,0.833400,0.291104,0.910500,0.887907,0.877285,0.881769,2000.000000,1775.213056,2884.747264
400,0.211100,0.196382,0.929000,0.908102,0.905313,0.906375,2000.000000,1775.213056,2884.890624


+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     0.1833 |
+-------------------------+------------+
| eval_accuracy           |     0.9260 |
+-------------------------+------------+
| eval_precision          |     0.8865 |
+-------------------------+------------+
| eval_recall             |     0.8834 |
+-------------------------+------------+
| eval_f1-score           |     0.8845 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_gpu_mem_mb         |  1775.0139 |
+-------------------------+------------+
| eval_cpu_mem_mb         |  3147.3500 |
+-------------------------+------------+
| eval_runtime            |     1.3571 |
+-------------------------+------------+
| eval_samples_per_second |  1473.6900 |
+-------------------------+------------+
| eval_steps_per_second   |    15.4740 |
+---------------

### 3. Обучим модель в режиме linear probing
В качестве головы был выбран дополнительный слой трансформера, поскольку данное предложение усилит контекстное представления
обеспечивая глубокую обработку скрытых состояний без дообучения всей модели. Думаю, дополнительное внимание лучше сработает, чем простой Linear(ReLU).

In [26]:
import torch
import torch.nn as nn
from transformers import BertConfig, BertTokenizer


class TransformerClassifierHead(nn.Module):
    def __init__(self, hidden_size, num_labels=6, num_layers=1, dropout=0.1):
        super(TransformerClassifierHead, self).__init__()
        
        self.config = BertConfig(
            hidden_size=hidden_size,
            num_attention_heads=12,
            intermediate_size=3072,
            num_hidden_layers=num_layers,
            hidden_dropout_prob=dropout
        )

        self.transformer = nn.TransformerEncoder(
            encoder_layer = nn.TransformerEncoderLayer(
                d_model=hidden_size,
                nhead=self.config.num_attention_heads,
                dim_feedforward=self.config.intermediate_size,
                dropout=dropout,
                activation="gelu",
                batch_first=True
            ),
            num_layers=num_layers
        )

        self.pooler = nn.Linear(hidden_size, hidden_size)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(dropout)
        self.classifier = nn.Linear(hidden_size, num_labels)

    
    def forward(self, hidden_states):
        if hidden_states.dim() == 2:
            hidden_states = hidden_states.unsqueeze(1)  # [batch, 1, hidden]
        
        transformer_out = self.transformer(hidden_states)  # [batch, 1, hidden]
        
        pooled = self.tanh(self.pooler(transformer_out.squeeze(1)))  # [batch, hidden]
        pooled = self.dropout(pooled)
        
        return self.classifier(pooled)


model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(labels)
)


hidden_size = model.config.hidden_size # 768

# Замораживаем слои
for param in model.parameters():
    param.requires_grad = False

# Заменяем классификатор
model.classifier = TransformerClassifierHead(hidden_size, len(labels))

trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total trainable parameters: {trainable_params:,}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Total trainable parameters: 7,683,078


In [27]:
# Все обучаемые параметры
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Обучается: {name}")

Обучается: classifier.transformer.layers.0.self_attn.in_proj_weight
Обучается: classifier.transformer.layers.0.self_attn.in_proj_bias
Обучается: classifier.transformer.layers.0.self_attn.out_proj.weight
Обучается: classifier.transformer.layers.0.self_attn.out_proj.bias
Обучается: classifier.transformer.layers.0.linear1.weight
Обучается: classifier.transformer.layers.0.linear1.bias
Обучается: classifier.transformer.layers.0.linear2.weight
Обучается: classifier.transformer.layers.0.linear2.bias
Обучается: classifier.transformer.layers.0.norm1.weight
Обучается: classifier.transformer.layers.0.norm1.bias
Обучается: classifier.transformer.layers.0.norm2.weight
Обучается: classifier.transformer.layers.0.norm2.bias
Обучается: classifier.pooler.weight
Обучается: classifier.pooler.bias
Обучается: classifier.classifier.weight
Обучается: classifier.classifier.bias


In [28]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=1000,
    learning_rate=1e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    logging_steps=1000,
    weight_decay=0.01,
    metric_for_best_model="f1-score",
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
    save_total_limit=2,
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score,Support,Gpu Mem Mb,Cpu Mem Mb
1000,1.433300,1.315592,0.496500,0.427630,0.284316,0.248327,2000.000000,989.931520,3161.006080


+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     1.2817 |
+-------------------------+------------+
| eval_accuracy           |     0.5010 |
+-------------------------+------------+
| eval_precision          |     0.4325 |
+-------------------------+------------+
| eval_recall             |     0.2838 |
+-------------------------+------------+
| eval_f1-score           |     0.2553 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_gpu_mem_mb         |   989.7324 |
+-------------------------+------------+
| eval_cpu_mem_mb         |  3166.9453 |
+-------------------------+------------+
| eval_runtime            |     1.5989 |
+-------------------------+------------+
| eval_samples_per_second |  1250.8730 |
+-------------------------+------------+
| eval_steps_per_second   |    13.1340 |
+---------------

### Обучите модель в режиме PEFT (Prompt tuning)
Выбор параметров для Prompt Tuning обусловлено самой задачей текстовой классификации:

- num_virtual_tokens=40 — достаточно длина промпта для того, чтобы уловить суть задачи.
- token_dim=768 — соответствует размеру эмбеддингов модели BERT-base.
- prompt_tuning_init="TEXT" с prompt_tuning_init_text="Classify the emotion..." — использование семантически осмысленной инициализации направляет модель с самого начала в сторону нужной задачи, ускоряя обучение.
- base_model_name_or_path и tokenizer_name_or_path — использование предварительно обученной модели BERT-base-uncased даёт сильную основу, адаптированную под задачи NLP.

В совокупности, эти параметры обеспечат баланс между управляемостью обучения и гибкостью (через виртуальные токены), снижая вероятность плохой начальной инициализации и повышая интерпретируемость подхода.

In [29]:
from peft import (
    PromptTuningConfig,
    get_peft_model,
    TaskType
)
from transformers import AutoModelForSequenceClassification

peft_config = PromptTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=40,  
    token_dim=768,          
    prompt_tuning_init="TEXT",
    prompt_tuning_init_text="Classify the emotion expressed in the following sentence:",
    base_model_name_or_path="bert-base-uncased",
    tokenizer_name_or_path="bert-base-uncased"
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(labels),
    return_dict=True
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 30,720 || all params: 109,517,574 || trainable%: 0.0281


In [30]:
training_args = TrainingArguments(
    output_dir="./peft_results",
    learning_rate=1e-4,          
    per_device_train_batch_size=32,
    num_train_epochs=5,         
    logging_steps=100,
    save_strategy="no"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ilike528149 (r1char9). Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.789000
200,1.750400
300,1.726700
400,1.711300


+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     1.6830 |
+-------------------------+------------+
| eval_accuracy           |     0.3475 |
+-------------------------+------------+
| eval_precision          |     0.0580 |
+-------------------------+------------+
| eval_recall             |     0.1667 |
+-------------------------+------------+
| eval_f1-score           |     0.0860 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_gpu_mem_mb         |   896.8817 |
+-------------------------+------------+
| eval_cpu_mem_mb         |  3208.1797 |
+-------------------------+------------+
| eval_runtime            |     2.9741 |
+-------------------------+------------+
| eval_samples_per_second |   672.4620 |
+-------------------------+------------+
| eval_steps_per_second   |    14.1220 |
+---------------

### 4. Lora
Выбор параметров LoRA обусловлен балансом между эффективностью, стабильностью обучения и вычислительными затратами:

- r=8 (ранг) — оптимален для захвата основных паттернов данных без избыточной параметризации (слишком низкий r теряет информацию, высокий — увеличивает риск переобучения).
- lora_alpha=16 — коэффициент масштабирования, согласованный с r (часто используют alpha = 2*r), чтобы сохранить соотношение влияния оригинальных и адаптивных весов.
- lora_dropout=0.1 — умеренная регуляризация для улучшения обобщающей способности.
- target_modules=["query", "value"] — слои, связанные с механизмом внимания, наиболее критичны для адаптации модели к задаче.
- bias="none" — исключение смещений уменьшает число параметров и упрощает обучение.

Параметры следуют рекомендациям оригинальной работы по LoRA и эмпирическим практикам для задач классификации (TaskType.SEQ_CLS), обеспечивая воспроизводимость и стабильность результатов.

In [31]:
from peft import (
    LoraConfig, 
    TaskType, 
    get_peft_model
)
from transformers import AutoModelForSequenceClassification

lora_config = LoraConfig(
        task_type=TaskType.SEQ_CLS,
        inference_mode=False,
        r=8,                # Ранг адаптеров
        lora_alpha=16,      # Коэффициент масштабирования
        lora_dropout=0.1,   # Дропаут для регуляризации
        target_modules=["query", "value"],  # Слои для применения LoRA
        bias="none"
    )


model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=len(labels)
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()


training_args = TrainingArguments(
    output_dir=f"./lora_results",
    learning_rate=3e-4,
    per_device_train_batch_size=32,
    num_train_epochs=5,
    eval_strategy="epoch",
    logging_steps=200
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics
)

trainer.train()
results = trainer.evaluate(tokenized_dataset["test"])
print(tabulate.tabulate(
    results.items(),
    headers=["Метрика", "Значение"],
    tablefmt="grid",
    floatfmt=".4f"
))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 299,526 || all params: 109,786,380 || trainable%: 0.2728


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1-score,Support,Gpu Mem Mb,Cpu Mem Mb
1,No log,1.091891,0.582500,0.282021,0.311175,0.242177,2000.000000,903.434240,3217.534976
2,No log,0.701087,0.739000,0.785870,0.569051,0.583188,2000.000000,903.434240,3217.756160
3,1.086800,0.516600,0.803000,0.801747,0.703821,0.727249,2000.000000,903.434240,3217.743872
4,1.086800,0.429801,0.852000,0.841210,0.785316,0.805047,2000.000000,903.434240,3218.006016
5,0.529600,0.407840,0.857500,0.848730,0.792769,0.813186,2000.000000,903.434240,3218.399232


+-------------------------+------------+
| Метрика                 |   Значение |
+=========================+============+
| eval_loss               |     0.3929 |
+-------------------------+------------+
| eval_accuracy           |     0.8640 |
+-------------------------+------------+
| eval_precision          |     0.8530 |
+-------------------------+------------+
| eval_recall             |     0.7706 |
+-------------------------+------------+
| eval_f1-score           |     0.7980 |
+-------------------------+------------+
| eval_support            |  2000.0000 |
+-------------------------+------------+
| eval_gpu_mem_mb         |   903.3011 |
+-------------------------+------------+
| eval_cpu_mem_mb         |  3218.3869 |
+-------------------------+------------+
| eval_runtime            |     5.4311 |
+-------------------------+------------+
| eval_samples_per_second |   368.2530 |
+-------------------------+------------+
| eval_steps_per_second   |     7.7330 |
+---------------

# Подводим итоги

In [32]:
from tabulate import tabulate



data = [
    ["Full Finetuning", 0.1862, 0.9255, 0.8858, 0.8784, 0.8811, 2000.0, 2.3639, 846.0490, 8.8840, 3.0],
    ["Linear Probing", 1.2817, 0.5010, 0.4325, 0.2838, 0.2553, 2000.0, 2.4988, 800.3830, 8.4040, 10.0],
    ["Prompt Tuning", 1.6830, 0.3475, 0.0580, 0.1667, 0.0860, 2000.0, 4.8374, 413.4450, 8.6820, 5.0],
    ["LoRA", 0.4083, 0.8535, 0.8219, 0.7636, 0.7829, 2000.0, 5.1118, 391.2530, 8.2160, 5.0]
]

headers = [
    "Метод", "eval_loss", "eval_accuracy", "eval_precision", 
    "eval_recall", "eval_f1", "eval_support", "eval_runtime", 
    "samples/sec", "steps/sec", "epoch"
]

print(tabulate(data, headers=headers, tablefmt="grid", floatfmt=".4f"))

+-----------------+-------------+-----------------+------------------+---------------+-----------+----------------+----------------+---------------+-------------+---------+
| Метод           |   eval_loss |   eval_accuracy |   eval_precision |   eval_recall |   eval_f1 |   eval_support |   eval_runtime |   samples/sec |   steps/sec |   epoch |
+=================+=============+=================+==================+===============+===========+================+================+===============+=============+=========+
| Full Finetuning |      0.1862 |          0.9255 |           0.8858 |        0.8784 |    0.8811 |      2000.0000 |         2.3639 |      846.0490 |      8.8840 |  3.0000 |
+-----------------+-------------+-----------------+------------------+---------------+-----------+----------------+----------------+---------------+-------------+---------+
| Linear Probing  |      1.2817 |          0.5010 |           0.4325 |        0.2838 |    0.2553 |      2000.0000 |         2.4988 |   

Full Finetuning и LoRA демонстрируют высокие показатели (F1-score ~0.88), что говорит об их эффективности.   

Правда LoRA уступает Full Finetuning, что делает его менее предпочительным выбором для данной задач (F1-score ~0.78).

Prompt Tuning (F1-score ~0.0860 ) и Linear Probing (F1-score ~0.26) значительно уступают в качестве.


На этих экспериментах наглядно видно, что дополнительное внимание всяко лучше, чем без него!!!